In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Embedding, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Parametry modelu
max_len = 20
embedding_dim = 64
num_heads = 4
ff_dim = 128
num_blocks = 2
dropout_rate = 0.1
vocab_size = 10000

# Prosty blok transformera
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Embedding i pozycjonowanie
class TokenAndPositionEmbedding(Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

# Budowa modelu
class TransformerSentimentModel(Model):
    def __init__(self, max_len, vocab_size, embed_dim, num_heads, ff_dim, num_blocks, rate=0.1):


        super(TransformerSentimentModel, self).__init__()
        self.embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)
        self.transformer_blocks = [
            TransformerBlock(embed_dim, num_heads, ff_dim, rate) for _ in range(num_blocks)
        ]
        self.global_avg_pool = tf.keras.layers.GlobalAveragePooling1D()
        self.dropout = Dropout(rate)
        self.classifier = Dense(3, activation="softmax")

    def call(self, inputs, training=False):  # Add training argument with default value
        x = self.embedding_layer(inputs)
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x, training=training)  # Pass training argument
        x = self.global_avg_pool(x)
        x = self.dropout(x, training=training)  # Pass training argument here as well
        return self.classifier(x)

# Przygotowanie danych
tokenizer = Tokenizer(num_words=vocab_size)
sentences = ["I love this product", "This is terrible", "Quite decent", "Amazing experience", "Horrible"]
labels = [2, 0, 1, 2, 0]  # 2: positive, 1: neutral, 0: negative

# Tokenizacja i padding
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
x_train = pad_sequences(sequences, maxlen=max_len)
y_train = tf.keras.utils.to_categorical(labels, num_classes=3)

# Inicjalizacja i kompilacja modelu
model = TransformerSentimentModel(max_len, vocab_size, embedding_dim, num_heads, ff_dim, num_blocks, dropout_rate)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Trenowanie modelu
model.fit(x_train, y_train, epochs=5, batch_size=2)

# Testowanie modelu
test_sentences = ["This was fantastic", "I don't like it", "It's okay"]
test_sequences = tokenizer.texts_to_sequences(test_sentences)
x_test = pad_sequences(test_sequences, maxlen=max_len)
predictions = model.predict(x_test)

for i, sentence in enumerate(test_sentences):
    print(f"'{sentence}' - Sentyment: {np.argmax(predictions[i])} (0: negatywny, 1: neutralny, 2: pozytywny)")


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.5750 - loss: 1.9962
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4500 - loss: 1.3733 
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4500 - loss: 1.2307 
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+00 - loss: 1.4330
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.1625 - loss: 1.5261
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
'This was fantastic' - Sentyment: 0 (0: negatywny, 1: neutralny, 2: pozytywny)
'I don't like it' - Sentyment: 0 (0: negatywny, 1: neutralny, 2: pozytywny)
'It's okay' - Sentyment: 0 (0: negatywny, 1: neutralny, 2: pozytywny)
